In [ ]:
https://www.amazon.in/s?k=MEN+WATCHES&crid=JAW72Y9RN42P&sprefix=men+watches%2Caps%2C394&ref=nb_sb_noss_1

In [1]:
#scrapping code for men shirt 

from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import logging
import concurrent.futures
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO)

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def scrape_product(shoe):
    try:
        name_element = WebDriverWait(shoe, 20).until(EC.presence_of_element_located((By.XPATH, ".//span[@class='a-size-base-plus a-color-base a-text-normal']")))
        name = name_element.text
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product name")
        name = "No name available"

    try:
        product_id = shoe.get_attribute("data-asin")
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product ID")
        product_id = ""

    try:
        price_element = shoe.find_element(By.XPATH, ".//span[@class='a-price-whole']")
        price = price_element.text if price_element else "0"
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product price")
        price = "No price available"

    try:
        product_url_element = shoe.find_element(By.XPATH, ".//a[@class='a-link-normal s-no-outline']")
        product_url = product_url_element.get_attribute("href") if product_url_element else ""
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product URL")
        product_url = ""

    try:
        brand_element = shoe.find_element(By.XPATH, ".//span[@class='a-size-base-plus a-color-base']")
        brand = brand_element.text.strip()
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product description")
        brand = "No description available"

    return {
        "product_name": name,
        "product_id": product_id,
        "price": price,
        "product_url": product_url,
        "brand_name": brand,
        "gender": "male"  # Hardcoded gender as female
    }

def main():
    # Use the Firefox driver
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)

    try:
        all_data = []  # List to store data from all pages

        for page_number in range(10, 80):  # Scraping 10 pages
            url = f"https://www.amazon.in/s?k=MEN+WATCHES&crid=JAW72Y9RN42P&sprefix=men+watches%2Caps%2C394&ref=nb_sb_noss_1&page={page_number}"
            logging.info(f"Scraping page {page_number}: {url}")

            # Setting random user agent for each request
            user_agent = get_random_user_agent()
            logging.info(f"Using user agent: {user_agent}")
            options.set_preference("general.useragent.override", user_agent)

            driver.get(url)
            driver.maximize_window()

            # Add a delay to allow the page to load
            time.sleep(2)

            # Adjusted XPath expression to capture all product elements
            shoes = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

            data = []

            # Concurrently scrape product information
            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = executor.map(scrape_product, shoes)

            # Collect results
            for result in results:
                data.append(result)
                print(json.dumps(result, indent=4))

            all_data.extend(data)

            logging.info(f"Page {page_number} data collected")

        # Save all data to a single JSON file
        with open("watch1.json", "w") as json_file:
            json.dump(all_data, json_file, indent=4)
        logging.info("All data saved to amazon_shoes_women.json")

        # Convert all data to a DataFrame
        df = pd.DataFrame(all_data)
        df.to_csv("watch1.csv", index=False)
        logging.info("All data saved to menshoes1.csv")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


INFO:root:Scraping page 10: https://www.amazon.in/s?k=MEN+WATCHES&crid=JAW72Y9RN42P&sprefix=men+watches%2Caps%2C394&ref=nb_sb_noss_1&page=10
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36
INFO:root:Page 10 data collected
INFO:root:Scraping page 11: https://www.amazon.in/s?k=MEN+WATCHES&crid=JAW72Y9RN42P&sprefix=men+watches%2Caps%2C394&ref=nb_sb_noss_1&page=11
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; rv:102.0) Gecko/20100101 Firefox/102.0
INFO:root:Page 11 data collected
INFO:root:Scraping page 12: https://www.amazon.in/s?k=MEN+WATCHES&crid=JAW72Y9RN42P&sprefix=men+watches%2Caps%2C394&ref=nb_sb_noss_1&page=12
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.36


{
    "product_name": "Men Blue Dial Analog Watch Analog Blue Dial Men's Watch-TH1710503",
    "product_id": "B0BPCRPPMH",
    "price": "15,250",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTozNzE4OTQ1MTc0NDU0MDUxOjE3MDg1NDAyODA6c3BfYXRmX25leHQ6MjAxNTE2NzgwMDU3OTg6OjA6Og&url=%2FTommy-Hilfiger-Analog-Watch-Watch-TH1710503%2Fdp%2FB0BPCRPPMH%2Fref%3Dsr_1_481%3Fcrid%3DJAW72Y9RN42P%26keywords%3DMEN%2BWATCHES%26qid%3D1708540280%26sprefix%3Dmen%2Bwatches%252Caps%252C394%26sr%3D8-481-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Tommy Hilfiger",
    "gender": "male"
}
{
    "product_name": "TB Timeless Collection Men's Analog White Dial Coloured Quartz Watch, Round Dial with 41 Case Width - BKPPGF2069I",
    "product_id": "B0B76ZF1R6",
    "price": "7,795",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTozNzE4OTQ1MTc0NDU0MDUxOjE3MDg1NDAyODA6c3BfYXRmX25leHQ6MjAxNjgxMDEyMTU4OTg6OjA6Og&url=%2FTed-Baker-Timeless-Collection-

INFO:root:Page 12 data collected
INFO:root:Scraping page 13: https://www.amazon.in/s?k=MEN+WATCHES&crid=JAW72Y9RN42P&sprefix=men+watches%2Caps%2C394&ref=nb_sb_noss_1&page=13
INFO:root:Using user agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.75 Safari/537.36


{
    "product_name": "Analog Grey Dial Men's Watch-TH1782346W",
    "product_id": "B0BR7R1F7T",
    "price": "9,336",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo3OTkwNzc3NDI1MTI3MDUxOjE3MDg1NDAyOTA6c3BfYXRmX25leHQ6MjAxNTE2NzgwMDY3OTg6OjA6Og&url=%2FTommy-Hilfiger-Analog-Grey-Watch-TH1782346W%2Fdp%2FB0BR7R1F7T%2Fref%3Dsr_1_529%3Fcrid%3DJAW72Y9RN42P%26keywords%3DMEN%2BWATCHES%26qid%3D1708540290%26sprefix%3Dmen%2Bwatches%252Caps%252C394%26sr%3D8-529-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Tommy Hilfiger",
    "gender": "male"
}
{
    "product_name": "Men's Square Watch, LED Backlight Large Screen Multi-Time Zone Men's Watch, Waterproof Analog Men's Digital Sports Watch - 2025",
    "product_id": "B0C3CL1WNM",
    "price": "1,709",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo3OTkwNzc3NDI1MTI3MDUxOjE3MDg1NDAyOTA6c3BfYXRmX25leHQ6MzAwMTA5NDk3NzI4ODMyOjowOjo&url=%2FSKMEI-Backlight-Multi-Time-Waterproof-Digi

INFO:root:Page 13 data collected
INFO:root:Scraping page 14: https://www.amazon.in/s?k=MEN+WATCHES&crid=JAW72Y9RN42P&sprefix=men+watches%2Caps%2C394&ref=nb_sb_noss_1&page=14
INFO:root:Using user agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/117.0


...

{
    "product_name": "Analog Gold Dial Men's Watch-TH1782417W",
    "product_id": "B0BR7PJ29F",
    "price": "10,076",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4MzQ4MzI3ODExMzg4MDUxOjE3MDg1NDA4MjE6c3BfYXRmX25leHQ6MjAxNTE2NzE5OTgxOTg6OjA6Og&url=%2FTommy-Hilfiger-Analog-Gold-Watch-TH1782417W%2Fdp%2FB0BR7PJ29F%2Fref%3Dsr_1_3649%3Fcrid%3DJAW72Y9RN42P%26keywords%3DMEN%2BWATCHES%26qid%3D1708540821%26sprefix%3Dmen%2Bwatches%252Caps%252C394%26sr%3D8-3649-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Tommy Hilfiger",
    "gender": "male"
}
{
    "product_name": "Analogue Men's Watch (Silver Dial Silver Colored Strap)",
    "product_id": "B07H672SX1",
    "price": "425",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4MzQ4MzI3ODExMzg4MDUxOjE3MDg1NDA4MjE6c3BfYXRmX25leHQ6MjAwMjEwNTk2MzkyMDg6OjA6Og&url=%2FAUREX-Analogue-Watch-Silver-Colored%2Fdp%2FB07H672SX1%2Fref%3Dsr_1_3650%3Fcrid%3DJAW72Y9RN42P%26keywords%3DMEN%2BW

INFO:root:Page 78 data collected
INFO:root:Scraping page 79: https://www.amazon.in/s?k=MEN+WATCHES&crid=JAW72Y9RN42P&sprefix=men+watches%2Caps%2C394&ref=nb_sb_noss_1&page=79
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0


{
    "product_name": "Watch for Men Fashion Business Men Watches Ultra-Thin Waterproof Chronograph Quartz Watches with Stainless Steel Band",
    "product_id": "B0BMQCN4C2",
    "price": "2,999",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyNTg4MTYyOTM2NDAxMDUxOjE3MDg1NDA4Mjk6c3BfYXRmX25leHQ6MzAwMTE5NTg0Mjc1NzMyOjowOjo&url=%2FNIBOSI-Analogue-Waterproof-Gentlemen-Birthday%2Fdp%2FB0BMQCN4C2%2Fref%3Dsr_1_3697%3Fcrid%3DJAW72Y9RN42P%26keywords%3DMEN%2BWATCHES%26qid%3D1708540829%26sprefix%3Dmen%2Bwatches%252Caps%252C394%26sr%3D8-3697-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "NIBOSI",
    "gender": "male"
}
{
    "product_name": "Watch for Men Diamond Business Dress Analog Quartz Stainless Steel Waterproof Luminous Date Two Tone Luxury Casual Wrist Watch",
    "product_id": "B0CQYR8DRR",
    "price": "2,805",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyNTg4MTYyOTM2NDAxMDUxOjE3MDg1NDA4Mjk6c3BfYXRmX25leHQ6MzA

INFO:root:Page 79 data collected
INFO:root:All data saved to amazon_shoes_women.json
INFO:root:All data saved to menshoes1.csv


{
    "product_name": "93 Day n Date Series Men's Classic Trending Style| Reliable Performance and Styish Men's Analog Watch - Perfect for Everyday Wear and Special Occasion",
    "product_id": "B0BW8XND46",
    "price": "429",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyMDYxMzkyMTczNDA2MDUxOjE3MDg1NDA4Mzc6c3BfYXRmX25leHQ6MzAwMDk2OTY1NTY0MzMyOjowOjo&url=%2FHMXT-93-Classic-Trending-Reliable-Performance%2Fdp%2FB0BW8XND46%2Fref%3Dsr_1_3745%3Fcrid%3DJAW72Y9RN42P%26keywords%3DMEN%2BWATCHES%26qid%3D1708540837%26sprefix%3Dmen%2Bwatches%252Caps%252C394%26sr%3D8-3745-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "HMXT",
    "gender": "male"
}
{
    "product_name": "Watches for Men Round Numerical Dial Men Watch with Silicon Strap Long Battery Life Stainless Steel Bracelet Black Chain with Adjustable Flexible Silicon Strap Watches for Boys.",
    "product_id": "B0B3HYZQNZ",
    "price": "349",
    "product_url": "https://www.amazon.in/sspa

In [2]:
import json

# Path to the JSON file
file_name = 'watch1.json'

# Number of rows to read
num_rows = 1000

# List to store the limited data
limited_data = []

# Read the JSON file and limit the number of rows
with open(file_name, 'r') as file:
    for index, line in enumerate(file):
        if index < num_rows:
            data = json.loads(line)
            limited_data.append(data)
        else:
            break

# Path to save the limited JSON file
output_file = 'watchfinal.json'

# Save the limited data as a JSON file
with open(output_file, 'w') as file:
    json.dump(limited_data, file, indent=4)

print(f"Saved the first {num_rows} rows of the JSON file as {output_file}")


JSONDecodeError: Expecting value: line 2 column 1 (char 2)